## Libraries

In [3]:
# General Imports
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import yaml
import numpy as np
import math
import csv
from glob import glob

from typing import Dict, Any, List, Optional

# Local Module Imports
import sys

sys.path.insert(0, "../src")

from log_setup import logger
from eval import (
    hausdorff_dist_wkt,
    iou_wkt,
    strided_temp_consistency,
    save_results_to_csv
)
from utils import get_files_dirs_ext

## Data Sources

In [4]:
# Base directory
BASE_DIR = os.path.dirname(os.getcwd())

# Load 'config' file
config_file = os.path.join(BASE_DIR, "config.yml")
with open(config_file, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

# Output directory
output_dir = "../outputs"

# Training set directory
train_dir = os.path.join(BASE_DIR, config["data"]["full"]["train_dir"])

# Testing set wkt file path
test_wkt_path = os.path.join(BASE_DIR, config["data"]["wkt"]["test_wkt"])

# Pyspatiotemporalgeom, Shape Based, CVAE output directories
pstg_dir = os.path.join(output_dir, "pstg")
shpe_dir = os.path.join(output_dir, "shape")
cvae_dir = os.path.join(output_dir, "CVAE")

### PNG to WKT conversion

In [3]:
wkt_files = get_files_dirs_ext([pstg_dir, shpe_dir, cvae_dir], "wkt", return_paths=True)
png_dirs = get_files_dirs_ext([pstg_dir, shpe_dir, cvae_dir], "png")
# show wkt files and png dirs on a pretty table with several rows

logger.info(f'{"WKT Files":<70} | {"PNG Directories":<70}')
logger.info(f"--" * 60)
for wkt_file, png_dir in zip(wkt_files, png_dirs):
    logger.info(f'{wkt_file:<70} | {png_dir:<70}')
if len(wkt_files) > len(png_dirs):
    for wkt_file in wkt_files[len(png_dirs):]:
        logger.info(f'{wkt_file:<70} |')
if len(png_dirs) > len(wkt_files):
    for png_dir in png_dirs[len(wkt_files):]:
        logger.info(f'{"":<70} | {png_dir:<70}')
logger.info(f"--" * 60)
logger.info(f"No. of WKT files: {len(wkt_files)}")
logger.info(f"No. of PNG dirs: {len(png_dirs)}")


INFO - WKT Files                                                              | PNG Directories                                                       
INFO - ------------------------------------------------------------------------------------------------------------------------
INFO - ../outputs/shape/interpol/sampled/WKT/shape_interpol_sampled.wkt       | ../outputs/CVAE/extrapol/90/sampled/PNG                               
INFO - ../outputs/CVAE/extrapol/70/full/WKT/file.wkt                          | ../outputs/CVAE/extrapol/30/unet/PNG                                  
INFO - ../outputs/CVAE/extrapol/70/sampled/WKT/file.wkt                       | ../outputs/CVAE/extrapol/10/unet/PNG                                  
INFO - ../outputs/pstg/interpol/full/WKT/pyspatial_interpol.wkt               | ../outputs/CVAE/interpol/sampled/PNG                                  
INFO - ../outputs/CVAE/extrapol/10/unet/WKT/extrapol_unet.wkt                 | ../outputs/CVAE/extrapol/90/full/PNG 

# Interpolation

In [5]:
# from wkt files, retrive interpol wkt files
interpol_wkt_files = []
for wkt_file in wkt_files:
    if "interpol" in wkt_file:
        interpol_wkt_files.append(wkt_file)
interpol_wkt_files.sort()
interpol_wkt_files

['../outputs/CVAE/interpol/full/WKT/file.wkt',
 '../outputs/CVAE/interpol/sampled/WKT/file.wkt',
 '../outputs/pstg/interpol/full/WKT/pyspatial_interpol.wkt',
 '../outputs/pstg/interpol/sampled/WKT/pyspatial_interpol_sampled.wkt',
 '../outputs/shape/interpol/full/WKT/shape_interpol.wkt',
 '../outputs/shape/interpol/sampled/WKT/shape_interpol_sampled.wkt']

## [BurnedAreaUAV](https://zenodo.org/records/7944963) Test Set Evalution

In [5]:
logger.info("Reading WKTs files...")
with open(os.path.join(BASE_DIR, config["data"]["wkt"]["train_wkt"]), "r") as f:
    train_wkt = f.read().splitlines()

with open(os.path.join(BASE_DIR, config["data"]["wkt"]["test_wkt"]), "r") as f:
    test_wkt = f.read().splitlines()

# evaluation set dictionary
eval_set_dict = {}
for i, wkt in enumerate(train_wkt):
    eval_set_dict[i*100] = wkt
for i, wkt in enumerate(test_wkt):
    eval_set_dict[(i*100)+20250] = wkt
eval_set_dict = dict(sorted(eval_set_dict.items()))

INFO - Reading WKTs files...


### Hausdorf Distance

In [5]:
# Create the pattern to match the WKT files
pattern_interpol = os.path.join(output_dir, "*", "interpol", "*", "WKT", "*.wkt")
interpol_wkt_files = glob(pattern_interpol)

percentages = [10, 30, 50, 70, 90]
extrapol_wkt_files = {}

for percentage in percentages:
    # Create the pattern to match the WKT files
    pattern = os.path.join(output_dir, "*", "extrapol", str(percentage), "*", "WKT", "*.wkt")
    # Store the file list in the dictionary using the percentage as the key
    extrapol_wkt_files[percentage] = glob(pattern)

In [6]:
# Create a dictionary to store the number frames to be evaluated
percentages = [10, 30, 50, 70, 90]
final_frames = {}
for percentage in percentages:
    frame_perc = math.ceil(22500 * (percentage / 100))
    final_frame = min(eval_set_dict.keys(), key=lambda x:abs(x-frame_perc))
    final_frames[percentage] = final_frame

final_frames

{10: 2200, 30: 6700, 50: 11200, 70: 15700, 90: 20250}

In [7]:
for percentage, final_frame in final_frames.items():
    # From eval set dict, choose all the keys >= final_frame
    eval_set_dict_perc = {k: v for k, v in eval_set_dict.items() if k >= final_frame}
    eval_set_perc = list(eval_set_dict_perc.keys())
    # Save the sorted dict to a WKT file
    with open(os.path.join(BASE_DIR, f"eval_{percentage}.wkt"), "w") as f:
        for key, poly in eval_set_dict_perc.items():
            f.write(str(poly))
            f.write("\n")

In [9]:
# Open the file in append mode
with open('interpol_hd.csv', 'a', newline='') as file:
    writer = csv.writer(file)

    # Write the header only if the file is empty
    if file.tell() == 0:
        writer.writerow(["wkt_file", "mean_hd", "hd_list"])

    for wkt_file in interpol_wkt_files:
        # test set frame indexes 
        idx = np.linspace(20250, 22450, 23).astype(int)
        mean_hd, hd_list = hausdorff_dist_wkt(gt_wkt=test_wkt_path, model_wkt=wkt_file, eval_idx=idx)

        logger.info(f"Hausdorff Distance for \033[1m{wkt_file}\033[0m")
        logger.info(f"Mean: {mean_hd:10.4f}, Std Dev.: {np.std(hd_list):10.4f}, Min: {np.min(hd_list):10.4f}, Max: {np.max(hd_list):10.4f}")
        logger.info("...........................................................................\n") 

        # Write the data
        writer.writerow([wkt_file, mean_hd, hd_list])

INFO - Reading WKTs files...
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/shape/interpol/sampled/WKT/shape_interpol_sampled.wkt
INFO - Mean:   108.3490, Std Dev.:     8.6310, Min:    91.9837, Max:   125.5388
INFO - ...........................................................................

INFO - Reading WKTs files...
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/shape/interpol/full/WKT/shape_interpol.wkt
INFO - Mean:    60.8145, Std Dev.:    33.3117, Min:    19.4440, Max:   117.0000
INFO - ...........................................................................

INFO - Reading WKTs files...
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/interpol/sampled/WKT/file.wkt
INFO - Mean:    87.7788, Std Dev.:     9.1669, Min:    76.5376, Max:   119.5492
INFO - ...........................................................................

INFO - Reading WKTs files...
INFO - Calcula

In [8]:
final_frames.items()

dict_items([(10, 2200), (30, 6700), (50, 11200), (70, 15700), (90, 20250)])

In [8]:
horinzont = [500, 1000, 1500, 2000]
periodicity = 100
# Create a dictionary to store the number frames to be evaluated
horinzont_frames = {}
for percentage, final_frame in final_frames.items():
    horinzont_frames[percentage] = {
        h: [
            frame
            for frame in range(
                final_frame + periodicity, final_frame + h + periodicity, periodicity
            )
        ]
        for h in horinzont
    }

In [9]:
for percentage in percentages:
    eval_wkt = os.path.join(BASE_DIR, f"eval_{percentage}.wkt")
    print(eval_wkt)

/home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/eval_10.wkt
/home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/eval_30.wkt
/home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/eval_50.wkt
/home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/eval_70.wkt
/home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/eval_90.wkt


## U-Net eval

In [9]:
# retrieve unet wkt files only
unet_wkt_files = {}
for percentage, wkt_files in extrapol_wkt_files.items():
    unet_wkt_files[percentage] = [wkt_file for wkt_file in wkt_files if "unet.wkt" in wkt_file]

In [30]:
for percentage in percentages:
    eval_wkt = os.path.join(BASE_DIR, f"eval_{percentage}.wkt")
    unet_wkt_file = os.path.abspath(unet_wkt_files[percentage][0])
    h_frames = horinzont_frames[percentage]
    print(eval_wkt, unet_wkt_file, h_frames)

    # Open the file in append mode
    with open("unet_{percentage}_hd.csv", "a", newline="") as file:
        writer = csv.writer(file)

        # Write the header only if the file is empty
        if file.tell() == 0:
            writer.writerow(
                [
                    "wkt_file",
                    "mean_hd",
                    "horinzon_500",
                    "horinzon_1000",
                    "horinzon_1500",
                    "horinzon_2000",
                ]
            )

        mean_hd, hd_list = hausdorff_dist_wkt(
            gt_wkt=eval_wkt, model_wkt=unet_wkt_file, eval_idx=h_frames
        )

        logger.info(f"Hausdorff Distance for \033[1m{unet_wkt_file}\033[0m")
        logger.info(
            f"Horizons: \n500: {hd_list[0]:10.4f}, \n1000: {hd_list[1]:10.4f}, \n1500: {hd_list[2]:10.4f}, \n2000: {hd_list[3]:10.4f}"
        )
        logger.info(
            "...........................................................................\n"
        )
        # Write the data
        writer.writerow(
            [unet_wkt_file, mean_hd, hd_list[0], hd_list[1], hd_list[2], hd_list[3]]
        )

INFO - Reading WKTs files...


/home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/eval_10.wkt /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/10/unet/WKT/extrapol_unet.wkt [2700, 3200, 3700, 4200]


WARNING - Number of polygons in ground truth (227) and model (4) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/10/unet/WKT/extrapol_unet.wkt
INFO - Horizons: 
500:    34.0147, 
1000:    27.0000, 
1500:    26.6271, 
2000:    35.9026
INFO - ...........................................................................

INFO - Reading WKTs files...


/home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/eval_30.wkt /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/30/unet/WKT/extrapol_unet.wkt [7200, 7700, 8200, 8700]


WARNING - Number of polygons in ground truth (182) and model (4) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/30/unet/WKT/extrapol_unet.wkt
INFO - Horizons: 
500:    33.0000, 
1000:    28.6078, 
1500:    33.1796, 
2000:    41.6580
INFO - ...........................................................................

INFO - Reading WKTs files...


...

WARNING - Number of polygons in ground truth (137) and model (4) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/50/unet/WKT/extrapol_unet.wkt
INFO - Horizons: 
500:    71.0000, 
1000:    71.0000, 
1500:    89.0000, 
2000:    88.0227
INFO - ...........................................................................

INFO - Reading WKTs files...


/home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/eval_70.wkt /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/70/unet/WKT/extrapol_unet.wkt [16200, 16700, 17200, 17700]


WARNING - Number of polygons in ground truth (92) and model (4) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/70/unet/WKT/extrapol_unet.wkt
INFO - Horizons: 
500:    38.6394, 
1000:    54.0000, 
1500:    56.1427, 
2000:    45.3431
INFO - ...........................................................................

INFO - Reading WKTs files...


/home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/eval_90.wkt /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/90/unet/WKT/extrapol_unet.wkt [20750, 21250, 21750, 22250]


WARNING - Number of polygons in ground truth (46) and model (4) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/90/unet/WKT/extrapol_unet.wkt
INFO - Horizons: 
500:    85.0000, 
1000:    43.0465, 
1500:    62.9682, 
2000:    41.9806
INFO - ...........................................................................



In [47]:

for percentage in percentages:
    eval_wkt = os.path.join(BASE_DIR, f"eval_{percentage}.wkt")
    unet_wkt_file = os.path.abspath(unet_wkt_files[percentage][0])
    h_frames = horinzont_frames[percentage]

    # Open the file in append mode
    with open(f"unet_{percentage}_hd_list.csv", "a", newline="") as file:
        writer = csv.writer(file)

        # Write the header only if the file is empty
        if file.tell() == 0:
            writer.writerow(
                [
                    "Percentage",
                    "Horizon",
                    "Mean",
                    "Std. Dev",
                    "Max",
                    "Min",
                    "List"
                ]
            )

        for horizon, frames in h_frames.items():
            mean_hd, hd_list = hausdorff_dist_wkt(
                gt_wkt=eval_wkt, model_wkt=unet_wkt_file, eval_idx=frames
            )

            std_hd = np.std(hd_list)
            max_hd = np.max(hd_list)
            min_hd = np.min(hd_list)

            logger.info(f"Hausdorff Distance for \033[1m{unet_wkt_file}\033[0m")
            logger.info(
                f"Horizon: {horizon}, Mean: {mean_hd:10.4f}, Std. Dev: {std_hd:10.4f}, Max: {max_hd:10.4f}, Min: {min_hd:10.4f}"
            )
            logger.info(
                "...........................................................................\n"
            )
            # Write the data
            writer.writerow(
                [percentage, horizon, mean_hd, std_hd, max_hd, min_hd, hd_list]
            )

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (227) and model (5) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/10/unet/WKT/extrapol_unet.wkt
INFO - Horizon: 500, Mean:    37.8044, Std. Dev:    11.3940, Max:    60.0083, Min:    29.0000
INFO - ...........................................................................

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (227) and model (10) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/10/unet/WKT/extrapol_unet.wkt
INFO - Horizon: 1000, Mean:    51.4301, Std. Dev:    23.5806, Max:   108.0185, Min:    29.0000
INFO - ...........................................................................

INFO - Reading WKTs files...
WARNING - Number of polygons in

In [46]:
for percentage in percentages:
    eval_wkt = os.path.join(BASE_DIR, f"eval_{percentage}.wkt")
    unet_wkt_file = os.path.abspath(unet_wkt_files[percentage][0])
    h_frames = horinzont_frames[percentage]

    # Open the file in append mode
    with open(f"unet_{percentage}_iou_list.csv", "a", newline="") as file:
        writer = csv.writer(file)

        # Write the header only if the file is empty
        if file.tell() == 0:
            writer.writerow(
                [
                    "Percentage",
                    "Horizon",
                    "Mean",
                    "Std. Dev",
                    "Max",
                    "Min",
                    "List"
                ]
            )

        for horizon, frames in h_frames.items():
            mean_hd, hd_list = iou_wkt(
                gt_wkt=eval_wkt, model_wkt=unet_wkt_file, eval_idx=frames
            )

            std_hd = np.std(hd_list)
            max_hd = np.max(hd_list)
            min_hd = np.min(hd_list)

            logger.info(f"IoU for \033[1m{unet_wkt_file}\033[0m")
            logger.info(
                f"Horizon: {horizon}, Mean: {mean_hd:10.4f}, Std. Dev: {std_hd:10.4f}, Max: {max_hd:10.4f}, Min: {min_hd:10.4f}"
            )
            logger.info(
                "...........................................................................\n"
            )
            # Write the data
            writer.writerow(
                [percentage, horizon, mean_hd, std_hd, max_hd, min_hd, hd_list]
            )

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (227) and model (5) do not match.
INFO - Calculating IoU...
INFO - IoU for /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/10/unet/WKT/extrapol_unet.wkt
INFO - Horizon: 500, Mean:     0.9300, Std. Dev:     0.0169, Max:     0.9536, Min:     0.9092
INFO - ...........................................................................

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (227) and model (10) do not match.
INFO - Calculating IoU...
INFO - IoU for /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/10/unet/WKT/extrapol_unet.wkt
INFO - Horizon: 1000, Mean:     0.9131, Std. Dev:     0.0210, Max:     0.9536, Min:     0.8918
INFO - ...........................................................................

INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (227) and model (15) do not match.
INFO - Calc

In [12]:
# Unet temporal consistency
strides = config["eval"]["strides"]
for percentage in percentages:
    eval_wkt = os.path.join(BASE_DIR, f"eval_{percentage}.wkt")
    unet_wkt_file = os.path.abspath(unet_wkt_files[percentage][0])
    h_frames = horinzont_frames[percentage]
    

    for horizon, frames in h_frames.items():
        tc_results = strided_temp_consistency(
            wkt_file=unet_wkt_file,
            strides=strides,
            interval=[final_frames[percentage], final_frames[percentage] + horizon],
        )
        # Save the results to a CSV file
        save_path = os.path.join(BASE_DIR, f"unet_{percentage}_tc_h_{horizon}")
        save_results_to_csv(results=tc_results, base_filename=save_path)

INFO - Reading WKT file /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/10/unet/WKT/extrapol_unet.wkt...
INFO - Validating polygons...
INFO - Calculating the temporal consistency with stride 1...


Number of polygons: 500
Valid strides: [1, 10, 100]


INFO - Calculating the temporal consistency with stride 10...


INFO - Calculating the temporal consistency with stride 100...


INFO - Saved results to /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/unet_10_tc_h_500.csv.
INFO - Reading WKT file /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/outputs/CVAE/extrapol/10/unet/WKT/extrapol_unet.wkt...


...

INFO - Validating polygons...
INFO - Calculating the temporal consistency with stride 1...


Number of polygons: 2000
Valid strides: [1, 10, 100]


INFO - Calculating the temporal consistency with stride 10...


INFO - Calculating the temporal consistency with stride 100...


INFO - Saved results to /home/tiagociic/Projectos/spatiotemporal-vae-reconstruction/unet_90_tc_h_2000.csv.


## Process unet tc

In [ ]:
for percentage, final_frame in final_frames.items():
    eval_wkt = os.path.join(BASE_DIR, f"eval_{percentage}.wkt")
    print(eval_wkt)
    print(percentage)
    extrapol_wkt_files_perc = extrapol_wkt_files[percentage]

    # Open the file in append mode
    with open(f'extrapol_{percentage}_hd.csv', 'a', newline='') as file:
        writer = csv.writer(file)

        # Write the header only if the file is empty
        if file.tell() == 0:
            writer.writerow(["wkt_file", "mean_hd", "hd_list"])

        for wkt_file in extrapol_wkt_files_perc:
            mean_hd, hd_list = hausdorff_dist_wkt(
                gt_wkt=eval_wkt,
                model_wkt=wkt_file,
                eval_idx=eval_set_perc[:-1],
            )

            logger.info(f"--------- Hausdorff Distance --------------------")
            logger.info(f"WKT File: {wkt_file}")
            logger.info(f"Mean Hausdorff Distance: {mean_hd:>10.4f}")
            logger.info(f"Standard Deviation:      {np.std(hd_list):>10.4f}")
            logger.info(f"Minimum:                 {np.min(hd_list):>10.4f}")
            logger.info(f"Maximum:                 {np.max(hd_list):>10.4f}")
            logger.info(f"-------------------------------------------------\n")

            # Write the data
            writer.writerow([wkt_file, mean_hd, hd_list])


In [15]:
# retrieve unet wkt files only
unet_wkt_files = {}
for percentage, wkt_files in extrapol_wkt_files.items():
    unet_wkt_files[percentage] = [wkt_file for wkt_file in wkt_files if "unet.wkt" in wkt_file]
unet_wkt_files

{10: ['../outputs/CVAE/extrapol/10/unet/WKT/extrapol_unet.wkt'],
 30: ['../outputs/CVAE/extrapol/30/unet/WKT/extrapol_unet.wkt'],
 50: ['../outputs/CVAE/extrapol/50/unet/WKT/extrapol_unet.wkt'],
 70: ['../outputs/CVAE/extrapol/70/unet/WKT/extrapol_unet.wkt'],
 90: ['../outputs/CVAE/extrapol/90/unet/WKT/extrapol_unet.wkt']}

In [ ]:
for wkt_file in sorted(wkt_files):
    # print like a table
    print(f"{wkt_file}")
print(" => No of WKT files: ", len(wkt_files))
print()

for png_dir in sorted(png_dirs):
    # print like a table
    print(f"{png_dir}")
print(" => No of PNG dirs: ", len(png_dirs))

../outputs/CVAE/extrapol/10/unet/WKT/extrapol_unet.wkt
../outputs/CVAE/extrapol/50/full/WKT/extrapol_full.wkt
../outputs/CVAE/extrapol/50/sampled/WKT/extrapol_sampled.wkt
../outputs/CVAE/extrapol/50/unet/WKT/extrapol_unet.wkt
../outputs/CVAE/extrapol/70/full/WKT/file.wkt
../outputs/CVAE/extrapol/70/sampled/WKT/file.wkt
../outputs/CVAE/extrapol/70/unet/WKT/extrapol_unet.wkt
../outputs/CVAE/extrapol/90/full/WKT/file.wkt
../outputs/CVAE/extrapol/90/sampled/WKT/file.wkt
../outputs/CVAE/extrapol/90/unet/WKT/extrapol_unet.wkt
../outputs/CVAE/interpol/full/WKT/file.wkt
../outputs/CVAE/interpol/sampled/WKT/file.wkt
../outputs/pstg/interpol/full/WKT/pyspatial_interpol.wkt
../outputs/pstg/interpol/sampled/WKT/pyspatial_interpol_sampled.wkt
../outputs/shape/interpol/full/WKT/shape_interpol.wkt
../outputs/shape/interpol/sampled/WKT/shape_interpol_sampled.wkt
 => No of WKT files:  16

../outputs/CVAE/extrapol/10/unet/PNG
../outputs/CVAE/extrapol/50/full/PNG
../outputs/CVAE/extrapol/50/sampled/PNG
.

In [11]:
eval90_wkt = os.path.join(BASE_DIR, "eval_90.wkt")

# Open the file in append mode
with open('extrapol_90_hd.csv', 'a', newline='') as file:
    writer = csv.writer(file)

    # Write the header only if the file is empty
    if file.tell() == 0:
        writer.writerow(["wkt_file", "mean_hd", "hd_list"])

    for wkt_file in extrapol_wkt_files_90:
        mean_hd, hd_list = hausdorff_dist_wkt(
            gt_wkt=eval90_wkt,
            model_wkt=wkt_file,
            eval_idx=eval_set_90[:-1],
        )

        logger.info(f"Hausdorff Distance for \033[1m{wkt_file}\033[0m")
        logger.info(
            f"Mean: {mean_hd:10.4f}, Std Dev.: {np.std(hd_list):10.4f}, Min: {np.min(hd_list):10.4f}, Max: {np.max(hd_list):10.4f}"
        )
        logger.info("\n")

        # Write the data
        writer.writerow([wkt_file, mean_hd, hd_list])

INFO - Reading WKTs files...
INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/extrapol/90/sampled/WKT/file.wkt
INFO - Hausdorff Distance for ../outputs/CVAE/extrapol/90/sampled/WKT/file.wkt
INFO - Mean:    95.0263, Std Dev.:    18.8012, Min:    72.3395, Max:   130.0000
INFO - Mean:    95.0263, Std Dev.:    18.8012, Min:    72.3395, Max:   130.0000
INFO - 

INFO - 

INFO - Reading WKTs files...
INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
INFO - Calculating Hausdorff Distance...
INFO - Calculating Hausdorff Distance...
INFO - Hausdorff Distance for ../outputs/CVAE/extrapol/90/full/WKT/f

### IoU 

In [12]:
# Open the file in append mode
with open("interpol_iou.csv", "a", newline="") as file:
    writer = csv.writer(file)

    # Write the header only if the file is empty
    if file.tell() == 0:
        writer.writerow(["wkt_file", "mean_hd", "hd_list"])

    for wkt_file in interpol_wkt_files:
        # test set frame indexes
        idx = np.linspace(20250, 22450, 23).astype(int)
        mean_iou, iou_list = iou_wkt(gt_wkt=test_wkt_path, model_wkt=wkt_file, eval_idx=idx)

        logger.info(f"IoU for \033[1m{wkt_file}\033[0m")
        logger.info(
            f"Mean: {mean_iou:10.4f}, Std Dev.: {np.std(iou_list):10.4f}, Min: {np.min(iou_list):10.4f}, Max: {np.max(iou_list):10.4f}"
        )
        logger.info(
            "...........................................................................\n"
        )

        # Write the data
        writer.writerow([wkt_file, mean_iou, iou_list])

INFO - Reading WKTs files...
INFO - Reading WKTs files...
INFO - Calculating IoU...
INFO - Calculating IoU...
INFO - IoU for ../outputs/shape/interpol/sampled/WKT/shape_interpol_sampled.wkt
INFO - IoU for ../outputs/shape/interpol/sampled/WKT/shape_interpol_sampled.wkt
INFO - Mean:     0.9296, Std Dev.:     0.0210, Min:     0.8867, Max:     0.9640
INFO - Mean:     0.9296, Std Dev.:     0.0210, Min:     0.8867, Max:     0.9640
INFO - ...........................................................................

INFO - ...........................................................................

INFO - Reading WKTs files...
INFO - Reading WKTs files...
INFO - Calculating IoU...
INFO - Calculating IoU...
INFO - IoU for ../outputs/shape/interpol/full/WKT/shape_interpol.wkt
INFO - IoU for ../outputs/shape/interpol/full/WKT/shape_interpol.wkt
INFO - Mean:     0.9589, Std Dev.:     0.0155, Min:     0.9253, Max:     0.9767
INFO - Mean:     0.9589, Std Dev.:     0.0155, Min:     0.9253, Max:     0

In [13]:
# Open the file in append mode
with open("extrapol_70_iou.csv", "a", newline="") as file:
    writer = csv.writer(file)

    # Write the header only if the file is empty
    if file.tell() == 0:
        writer.writerow(["wkt_file", "mean_hd", "hd_list"])

    for wkt_file in extrapol_wkt_files_70:
        mean_iou, iou_list = iou_wkt(
            gt_wkt=eval70_wkt,
            model_wkt=wkt_file,
            eval_idx=eval_set_70[:-1],
        )

        logger.info(f"IoU for \033[1m{wkt_file}\033[0m")
        logger.info(
            f"Mean: {mean_iou:10.4f}, Std Dev.: {np.std(iou_list):10.4f}, Min: {np.min(iou_list):10.4f}, Max: {np.max(iou_list):10.4f}"
        )
        logger.info("...........................................................................\n")

        # Write the data
        writer.writerow([wkt_file, mean_iou, iou_list])

INFO - Reading WKTs files...
INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (90) do not match.
WARNING - Number of polygons in ground truth (91) and model (90) do not match.
INFO - Calculating IoU...
INFO - Calculating IoU...
INFO - IoU for ../outputs/CVAE/extrapol/70/sampled/WKT/file.wkt
INFO - IoU for ../outputs/CVAE/extrapol/70/sampled/WKT/file.wkt
INFO - Mean:     0.6467, Std Dev.:     0.0370, Min:     0.5987, Max:     0.7244
INFO - Mean:     0.6467, Std Dev.:     0.0370, Min:     0.5987, Max:     0.7244
INFO - ...........................................................................

INFO - ...........................................................................

INFO - Reading WKTs files...
INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (90) do not match.
WARNING - Number of polygons in ground truth (91) and model (90) do not match.
INFO - Calculating IoU...
INFO - Calculating IoU...
INFO - I

In [14]:
# Open the file in append mode
with open("extrapol_90_iou.csv", "a", newline="") as file:
    writer = csv.writer(file)

    # Write the header only if the file is empty
    if file.tell() == 0:
        writer.writerow(["wkt_file", "mean_hd", "hd_list"])


    for wkt_file in extrapol_wkt_files_90:
        mean_iou, iou_list = iou_wkt(
            gt_wkt=eval90_wkt,
            model_wkt=wkt_file,
            eval_idx=eval_set_90[:-1],
        )

        logger.info(f"IoU for \033[1m{wkt_file}\033[0m")
        logger.info(
            f"Mean: {mean_iou:10.4f}, Std Dev.: {np.std(iou_list):10.4f}, Min: {np.min(iou_list):10.4f}, Max: {np.max(iou_list):10.4f}"
        )
        logger.info("...........................................................................\n")

        # Write the data
        writer.writerow([wkt_file, mean_iou, iou_list])

INFO - Reading WKTs files...
INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
INFO - Calculating IoU...
INFO - Calculating IoU...
INFO - IoU for ../outputs/CVAE/extrapol/90/sampled/WKT/file.wkt
INFO - IoU for ../outputs/CVAE/extrapol/90/sampled/WKT/file.wkt
INFO - Mean:     0.8697, Std Dev.:     0.0385, Min:     0.8119, Max:     0.9231
INFO - Mean:     0.8697, Std Dev.:     0.0385, Min:     0.8119, Max:     0.9231
INFO - ...........................................................................

INFO - ...........................................................................

INFO - Reading WKTs files...
INFO - Reading WKTs files...
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
WARNING - Number of polygons in ground truth (91) and model (43) do not match.
INFO - Calculating IoU...
INFO - Calculating IoU...
INFO - I

## Temporal Consistency

In [12]:

# calculate temporal consistency for each model in each mode
for wkt_file in wkt_files:
    tc = strided_temp_consistency(
        wkt_file=wkt_file,
        num_polygons=[0,22500],
        strides=strides
    )
    save_path = os.path.join(os.path.dirname(wkt_file), "tc")
    save_results_to_csv(results=tc, base_filename=save_path)

DEBUG - Reading WKT file...
DEBUG - Validating polygons...
INFO - Calculating the temporal consistency with stride 1...
INFO - Calculating the temporal consistency with stride 10...
INFO - Calculating the temporal consistency with stride 100...
INFO - Calculating the temporal consistency with stride 1000...
INFO - Calculating the temporal consistency with stride 10000...
DEBUG - Reading WKT file...
DEBUG - Validating polygons...
INFO - Calculating the temporal consistency with stride 1...
INFO - Calculating the temporal consistency with stride 10...
INFO - Calculating the temporal consistency with stride 100...
INFO - Calculating the temporal consistency with stride 1000...
INFO - Calculating the temporal consistency with stride 10000...
DEBUG - Reading WKT file...
DEBUG - Validating polygons...
INFO - Calculating the temporal consistency with stride 1...
INFO - Calculating the temporal consistency with stride 10...
INFO - Calculating the temporal consistency with stride 100...
